# Virtual Concierge

## Compile Pretrained MXNET model with NEO

In this notebook we will download a pre-trained MXNET model and compile for `ml_m4` and `deeplens` targets.

In [ ]:
# Download pre-trained model if haven't already created from previous notebook
import os

if not os.path.exists('model.tar.gz'):
    !aws s3 cp s3://deeplens-virtual-concierge-model/mobilefacenet/model.tar.gz .

## Invoke Neo Compilation API

We then forward the model artifact to Neo Compilation API:

In [ ]:
import boto3
import sagemaker
import time
from sagemaker.utils import name_from_base

role = sagemaker.get_execution_role()
sess = sagemaker.Session()
region = sess.boto_region_name
bucket = sess.default_bucket()

compilation_job_name = name_from_base('virtual-concierge')

model_key = '{}/model/model.tar.gz'.format(compilation_job_name)
model_path = 's3://{}/{}'.format(bucket, model_key)
boto3.resource('s3').Bucket(bucket).upload_file('model.tar.gz', model_key)

sm_client = boto3.client('sagemaker')
data_shape = '{"data":[1,3,112,112]}'
target_device = 'ml_m4'
framework = 'MXNET'
framework_version = '1.2'
compiled_model_path = 's3://{}/{}/output'.format(bucket, compilation_job_name)

In [ ]:
response = sm_client.create_compilation_job(
    CompilationJobName=compilation_job_name,
    RoleArn=role,
    InputConfig={
        'S3Uri': model_path,
        'DataInputConfig': data_shape,
        'Framework': framework
    },
    OutputConfig={
        'S3OutputLocation': compiled_model_path,
        'TargetDevice': target_device
    },
    StoppingCondition={
        'MaxRuntimeInSeconds': 300
    }
)
print(response)

# Poll every 30 sec
while True:
    response = sm_client.describe_compilation_job(CompilationJobName=compilation_job_name)
    if response['CompilationJobStatus'] == 'COMPLETED':
        break
    elif response['CompilationJobStatus'] == 'FAILED':
        raise RuntimeError('Compilation failed')
    print('Compiling ...')
    time.sleep(30)
print('Done!')

# Extract compiled model artifact
compiled_model_path = response['ModelArtifacts']['S3ModelArtifacts']

## Create prediction endpoint

To create a prediction endpoint, we first specify two additional functions, to be used with Neo Deep Learning Runtime:

* `neo_preprocess(payload, content_type)`: Function that takes in the payload and Content-Type of each incoming request and returns a NumPy array. Here, the payload is byte-encoded NumPy array, so the function simply decodes the bytes to obtain the NumPy array.
* `neo_postprocess(result)`: Function that takes the prediction results produced by Deep Learining Runtime and returns the response body

In [ ]:
!pygmentize predict.py

Upload the Python script containing the two functions to S3:

In [ ]:
import tarfile

source_key = '{}/source/sourcedir.tar.gz'.format(compilation_job_name)
source_path = 's3://{}/{}'.format(bucket, source_key)

with tarfile.open('sourcedir.tar.gz', 'w:gz') as f:
    f.add('predict.py')

boto3.resource('s3').Bucket(bucket).upload_file('sourcedir.tar.gz', source_key)

We then create a SageMaker model record:

In [ ]:
from sagemaker.model import NEO_IMAGE_ACCOUNT
from sagemaker.fw_utils import create_image_uri

model_name = name_from_base('virtual-concierge') + target_device.replace('_', '-')

image_uri = create_image_uri(region, 'neo-' + framework.lower(), target_device.replace('_', '.'),
                             framework_version, py_version='py3', account=NEO_IMAGE_ACCOUNT[region])

response = sm_client.create_model(
    ModelName=model_name,
    PrimaryContainer={
        'Image': image_uri,
        'ModelDataUrl': compiled_model_path,
        'Environment': { 'SAGEMAKER_SUBMIT_DIRECTORY': source_path }
    },
    ExecutionRoleArn=role
)
print(response)

Then we create an Endpoint Configuration:

In [ ]:
config_name = model_name

response = sm_client.create_endpoint_config(
    EndpointConfigName=config_name,
    ProductionVariants=[
        {
            'VariantName': 'default-variant-name',
            'ModelName': model_name,
            'InitialInstanceCount': 1,
            'InstanceType': 'ml.m4.xlarge',
            'InitialVariantWeight': 1.0
        },
    ],
)
print(response)

Finally, we create an Endpoint:

In [ ]:
endpoint_name = model_name

response = sm_client.create_endpoint(
    EndpointName=endpoint_name,
    EndpointConfigName=config_name,
)
print(response)

print('Creating endpoint ...')
sm_client.get_waiter('endpoint_in_service').wait(EndpointName=endpoint_name)

response = sm_client.describe_endpoint(EndpointName=endpoint_name)
print(response)

## Send requests

Download a sample picture, detect the first face

In [ ]:
import boto3
import json
import base64
import io
import PIL.Image

s3 = boto3.resource('s3')

# Read image from s3
image = {
    'S3Object': {
        'Bucket': 'aiml-lab-sagemaker',
        'Name': 'politicians/politicians2.jpg'
    }
}

image_object = s3.Object(image['S3Object']['Bucket'] , image['S3Object']['Name'])
payload = image_object.get()['Body'].read()

Send the payload to the endpoint, and output the face embedding response

In [ ]:
%%time

import json
import numpy as np

sm_runtime = boto3.Session().client('sagemaker-runtime')

response = sm_runtime.invoke_endpoint(EndpointName=endpoint_name,
                                      ContentType='application/x-image',
                                      Body=payload)
print(response)

In [ ]:
np.array(json.loads(response['Body'].read().decode()))[:10]

Pre-process the payload to add the BoundaryBox and Roll

In [ ]:
import json
import base64
import numpy as np

rekognition = boto3.client('rekognition')
    
# Call rekognition to get bbox
ret = rekognition.detect_faces(
    Image={
        'Bytes': payload
    },
    Attributes=['DEFAULT'],
)

bbox, roll = ret['FaceDetails'][0]['BoundingBox'],  round(ret['FaceDetails'][0]['Pose']['Roll']/90)*90
print(bbox, roll)

event = { 
    'Image': { 'Bytes': str(base64.b64encode(payload), 'utf-8') },
    'BoundingBox': bbox, 
    'Roll': roll,
}

In [ ]:
%%time

sm_runtime = boto3.Session().client('sagemaker-runtime')

response = sm_runtime.invoke_endpoint(EndpointName=endpoint_name,
                                      ContentType='application/json',
                                      Body=json.dumps(event))
print(response)

## Clean up

Tear down the Neo endpoint, configuration and model

In [ ]:
sm_client.delete_endpoint(EndpointName=endpoint_name)

In [ ]:
sm_client.delete_endpoint_config(EndpointConfigName=config_name)

In [ ]:
sm_client.delete_model(ModelName=model_name)